In [ ]:
!rm customers.sqlite food_new.csv

In [5]:
# Dependencies
import pandas as pd
import numpy as np
import os

# Data Cleaning

In [6]:
# Path of the CSV file
csvfile = "restaurant_updated.csv"

In [7]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)

In [8]:
# Preview DataFrame
# Note that some rows are missing gender
df.head()

,id,the_geom,city,latitude,longitude,invalid_the_geom,storenumber,storetype,address,cartodb_id,...,Population_2014,FF_per_capita_2009,FF_per_capita_2014,p_change_FF_Per_Capita,Grocery_per_capita_2009,Grocery_per_capita_2014,p_change_in_grocery_per_capita,p_change_in_Access,p_Change_in_Obesity,p_change_in_income
0,1,0101000020E6100000541C075E2DEB60C09F909DB7B187...,SITKA,57.060111,-135.349288,Point,7972,FREESTANDING,913 HALIBUT POINT RD,14168,...,8883,0.0004504,0.00045,-0.02,0.0004504,0.0004503,-0.02,0,-3.37,18.06
1,2,0101000020E610000056F31C916FD260C0C8EC2C7AA72E...,JUNEAU,58.364486,-134.576119,Point,6123,FREESTANDING,2285 TROUT ST,14169,...,32511,0.000703437,0.00095,35.55,0.000223821,0.00018455,-17.54,0,9.16,18.68
2,3,0101000020E61000009E42AED4B3425DC0F9156BB8C835...,CLARKSTON,46.420188,-117.042226,Point,10610,FREESTANDING,350 BRIDGE ST,8632,...,22177,0.000416223,0.00045,8.34,0.000231235,0.00022546,-2.5,0,14.38,18.39
3,4,0101000020E610000032022A1C41D25DC0B7ED7BD45F21...,RICHLAND,46.260737,-119.285224,Point,6556,FREESTANDING,101 D WELLSIAN WAY,13945,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03
4,5,0101000020E6100000624B8FA67AF25DC0A48CB800341C...,PROSSER,46.220337,-119.788736,Point,11889,FREESTANDING,103 MERLOT DR,13886,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03


In [9]:
df = df.drop(['address', 'phone'], 1)

In [10]:
df.head()

,id,the_geom,city,latitude,longitude,invalid_the_geom,storenumber,storetype,cartodb_id,state,...,Population_2014,FF_per_capita_2009,FF_per_capita_2014,p_change_FF_Per_Capita,Grocery_per_capita_2009,Grocery_per_capita_2014,p_change_in_grocery_per_capita,p_change_in_Access,p_Change_in_Obesity,p_change_in_income
0,1,0101000020E6100000541C075E2DEB60C09F909DB7B187...,SITKA,57.060111,-135.349288,Point,7972,FREESTANDING,14168,AK,...,8883,0.0004504,0.00045,-0.02,0.0004504,0.0004503,-0.02,0,-3.37,18.06
1,2,0101000020E610000056F31C916FD260C0C8EC2C7AA72E...,JUNEAU,58.364486,-134.576119,Point,6123,FREESTANDING,14169,AK,...,32511,0.000703437,0.00095,35.55,0.000223821,0.00018455,-17.54,0,9.16,18.68
2,3,0101000020E61000009E42AED4B3425DC0F9156BB8C835...,CLARKSTON,46.420188,-117.042226,Point,10610,FREESTANDING,8632,WA,...,22177,0.000416223,0.00045,8.34,0.000231235,0.00022546,-2.5,0,14.38,18.39
3,4,0101000020E610000032022A1C41D25DC0B7ED7BD45F21...,RICHLAND,46.260737,-119.285224,Point,6556,FREESTANDING,13945,WA,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03
4,5,0101000020E6100000624B8FA67AF25DC0A48CB800341C...,PROSSER,46.220337,-119.788736,Point,11889,FREESTANDING,13886,WA,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03


In [11]:
new_csv = "cleaned.csv"
df.to_csv(new_csv, index=False)

# Database Creation

In [12]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [13]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///foodstore_seven.sqlite")

In [14]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [15]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'food_seven'
    
    id=Column(Integer,primary_key=True)
    the_geom=Column(Text)
    city=Column(Text)
    latitude=Column(Float)
    longitude=Column(Float)
    invalid_the_geom=Column(Text)
    storenumber=Column(Float)
    storetype=Column(String)
    #address=Column(String)
    cartodb_id=Column(Float)
    state=Column(Text)
    zip=Column(Float)
    #phone=Column(Float)
    playplace=Column(Text)
    drivethru=Column(Text)
    archcard=Column(Text)
    freewifi=Column(Text)
    storeurl=Column(Text)
    cartodb_georef_status=Column(Float)
    COUNTY=Column(Text)
    Grocery_2009=Column(Float)
    Convenience_2009=Column(Float)
    Fast_Food_Restaurants_2009=Column(Float)
    Adult_Diabetes_p_2009=Column(Float)
    Adult_Obsesity_p_2009=Column(Float)
    Median_Household_Income_2009=Column(Float)
    Limited_Access_Populationp_2009=Column(Float)
    Limited_Access_Population_2009=Column(Float)
    Population_2009=Column(Float)
    Grocery_2014=Column(Float)
    Convenience_2014=Column(Float)
    Fast_Food_Restaurants_2014=Column(Float)
    Adult_Diabetes_2014=Column(Float)
    Adult_Obesity_2014=Column(Float)
    Median_Household_Income_2014=Column(Float)
    Limited_Access_p_2014=Column(Float)
    Limited_Access_2014=Column(Float)
    Population_2014=Column(Float)
    FF_per_capita_2009=Column(Float)
    FF_per_capita_2014=Column(Float)
    p_change_FF_Per_Capita=Column(Float)
    Grocery_per_capita_2009=Column(Float)
    Grocery_per_capita_2014=Column(Float)
    p_change_in_grocery_per_capita=Column(Float)
    p_change_in_Access=Column(Float)
    p_Change_in_Obesity=Column(Float)
    p_change_in_income=Column(Float)
    
  

In [16]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [17]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(new_csv)

In [18]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [19]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

#data[:1]

[{'id': 1, 'the_geom': '0101000020E6100000541C075E2DEB60C09F909DB7B1874C40', 'city': 'SITKA', 'latitude': 57.060111, 'longitude': -135.349288, 'invalid_the_geom': 'Point', 'storenumber': 7972, 'storetype': 'FREESTANDING', 'cartodb_id': 14168, 'state': 'AK', 'zip': 99835, 'playplace': False, 'drivethru': True, 'archcard': True, 'freewifi': True, 'storeurl': 'http://www.mcalaska.com/7972', 'cartodb_georef_status': True, 'COUNTY': 'Sitka', 'Grocery_2009': 4, 'Convenience_2009': 2, 'Fast_Food_Restaurants_2009': 4, 'Adult_Diabetes_p_2009': 7.0, 'Adult_Obsesity_p_2009': 29.7, 'Median_Household_Income_2009': 58000, 'Limited_Access_Populationp_2009': 18, 'Limited_Access_Population_2009': 1617, 'Population_2009': 8881, 'Grocery_2014': 4, 'Convenience_2014': 2, 'Fast_Food_Restaurants_2014': 4, 'Adult_Diabetes_2014': 6.7, 'Adult_Obesity_2014': 28.7, 'Median_Household_Income_2014': 68472, 'Limited_Access_p_2014': 18, 'Limited_Access_2014': 1632, 'Population_2014': 8883, 'FF_per_capita_2009': 0.000

In [20]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [21]:
# Save the reference to the `customers` table as a variable called `food`
table = sqlalchemy.Table('food_seven', metadata, autoload=True)

In [22]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [23]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [24]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from food_seven limit 5").fetchall()

[(1, '0101000020E6100000541C075E2DEB60C09F909DB7B1874C40', 'SITKA', 57.060111, -135.349288, 'Point', 7972.0, 'FREESTANDING', 14168.0, 'AK', 99835.0, None, '0', '1', '1', '1', 'http://www.mcalaska.com/7972', 1.0, 'Sitka', 4.0, 2.0, 4.0, 7.0, 29.7, 58000.0, 18.0, 1617.0, 8881.0, 4.0, 2.0, 4.0, 6.7, 28.7, 68472.0, 18.0, 1632.0, 8883.0, 0.00045039999999999994, 0.00045, -0.02, 0.00045039999999999994, 0.0004503, -0.02, 0.0, -3.37, 18.06),
 (2, '0101000020E610000056F31C916FD260C0C8EC2C7AA72E4D40', 'JUNEAU', 58.364486, -134.576119, 'Point', 6123.0, 'FREESTANDING', 14169.0, 'AK', 99801.0, None, '1', '1', '1', '1', 'http://www.mcalaska.com/6123', 1.0, 'Juneau', 7.0, 4.0, 22.0, 5.5, 26.2, 69844.0, 32.0, 10138.0, 31275.0, 6.0, 9.0, 31.0, 5.9, 28.6, 82892.0, 32.0, 10063.0, 32511.0, 0.000703437, 0.00095, 35.55, 0.000223821, 0.00018455, -17.54, 0.0, 9.16, 18.68),
 (3, '0101000020E61000009E42AED4B3425DC0F9156BB8C8354740', 'CLARKSTON', 46.420188, -117.042226, 'Point', 10610.0, 'FREESTANDING', 8632.0, '

In [25]:
conn.execute("select * from food_seven where state = 'AL' ").fetchall()

[(2820, '0101000020E61000006A17D34CF75E55C06EA46C91B44D4040', 'AUBURN', 32.607073, -85.48384399999999, 'Point', 3427.0, 'FREESTANDING', 4413.0, 'AL', 36830.0, None, '0', '1', '1', '1', 'http://www.mcalabama.com/3427', 1.0, 'Lee', 13.0, 74.0, 108.0, 10.2, 30.0, 40102.0, 31.0, 43795.0, 140247.0, 17.0, 76.0, 128.0, 9.5, 29.0, 46442.0, 26.0, 36836.0, 154277.0, 0.00077007, 0.0008300000000000001, 7.74, 9.27e-05, 0.00011019, 18.88, -16.13, -3.33, 15.81),
 (2821, '0101000020E6100000D061BEBC005B55C0616BB6F292514040', 'OPELIKA', 32.637297, -85.42192, 'Point', 28375.0, 'WAL*MART', 4215.0, 'AL', 36801.0, None, '0', '0', '1', '1', 'http://www.mcalabama.com/28375', 1.0, 'Lee', 13.0, 74.0, 108.0, 10.2, 30.0, 40102.0, 31.0, 43795.0, 140247.0, 17.0, 76.0, 128.0, 9.5, 29.0, 46442.0, 26.0, 36836.0, 154277.0, 0.00077007, 0.0008300000000000001, 7.74, 9.27e-05, 0.00011019, 18.88, -16.13, -3.33, 15.81),
 (2822, '0101000020E610000009C3802557EF55C0FD15325706ED3F40', 'THOMASVILLE', 31.925878000000004, -87.73969

In [21]:
############## Import Libraries #####################
import datetime as dt
import numpy as np
import pandas as pd
import os

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc


from flask import (
    Flask, 
    render_template, 
    jsonify, 
    request, 
    redirect)
################# Flask Setup ##########################

app = Flask(__name__)

#################################################
# sqlite : connect to the existing database
#################################################

engine = create_engine("sqlite:///foodstore_seven.sqlite", echo=False)
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
Food = Base.classes.food_seven

session = Session(engine)



In [22]:
stmt = session.query(Food).statement
df = pd.read_sql_query(stmt, session.bind)
df.iloc[:,9]

0        AK
1        AK
2        WA
3        WA
4        WA
5        WA
6        WA
7        WA
8        WA
9        WA
10       WA
11       WA
12       WA
13       WA
14       WA
15       WA
16       WA
17       WA
18       WA
19       WA
20       WA
21       WA
22       WA
23       WA
24       WA
25       WA
26       WA
27       WA
28       WA
29       WA
         ..
12012    MA
12013    MA
12014    MA
12015    MA
12016    MA
12017    MA
12018    MA
12019    MA
12020    MA
12021    MA
12022    MA
12023    MA
12024    MA
12025    MA
12026    MA
12027    MA
12028    MA
12029    MA
12030    MA
12031    MA
12032    MA
12033    MA
12034    MA
12035    MA
12036    MA
12037    MA
12038    MA
12039    MA
12040    MA
12041    MA
Name: state, Length: 12042, dtype: object

In [23]:
stmt = Base.classes.food_seven.__table__.columns.keys()
#sample_list = stmt[9:10]
#return jsonify(sample_list[9:10])
stmt
#sample_list

['id',
 'the_geom',
 'city',
 'latitude',
 'longitude',
 'invalid_the_geom',
 'storenumber',
 'storetype',
 'cartodb_id',
 'state',
 'zip',
 'phone',
 'playplace',
 'drivethru',
 'archcard',
 'freewifi',
 'storeurl',
 'cartodb_georef_status',
 'COUNTY',
 'Grocery_2009',
 'Convenience_2009',
 'Fast_Food_Restaurants_2009',
 'Adult_Diabetes_p_2009',
 'Adult_Obsesity_p_2009',
 'Median_Household_Income_2009',
 'Limited_Access_Populationp_2009',
 'Limited_Access_Population_2009',
 'Population_2009',
 'Grocery_2014',
 'Convenience_2014',
 'Fast_Food_Restaurants_2014',
 'Adult_Diabetes_2014',
 'Adult_Obesity_2014',
 'Median_Household_Income_2014',
 'Limited_Access_p_2014',
 'Limited_Access_2014',
 'Population_2014',
 'FF_per_capita_2009',
 'FF_per_capita_2014',
 'p_change_FF_Per_Capita',
 'Grocery_per_capita_2009',
 'Grocery_per_capita_2014',
 'p_change_in_grocery_per_capita',
 'p_change_in_Access',
 'p_Change_in_Obesity',
 'p_change_in_income']

In [24]:
@app.route('/state')
def states():

    stmt = session.query(Food).statement
    df = pd.read_sql_query(stmt, session.bind)
    df.set_index('id', inplace=True)
    s = df['state'].unique()

    print(list(s.columns))
    # Return a list of the column names (sample names)
    return jsonify(list(s.columns))

In [25]:

# stmt = session.query(Food).statement
# df = pd.read_sql_query(stmt, session.bind)
# df.set_index('id', inplace=True)
# s = df.groupby(['state'])['zip'].count().to_frame()
# s.index.tolist()

stmt = session.query(Food).statement
df = pd.read_sql_query(stmt, session.bind)
df.set_index('id', inplace=True)
s = df.groupby(['state'])['zip'].count().to_frame()
the_states = s.index.tolist()
the_states

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

In [27]:
state_data = session.query(Food).first()
state_data.__dict__

{'Adult_Diabetes_2014': 6.7,
 'Adult_Diabetes_p_2009': 7.0,
 'Adult_Obesity_2014': 28.7,
 'Adult_Obsesity_p_2009': 29.7,
 'COUNTY': 'Sitka',
 'Convenience_2009': 2.0,
 'Convenience_2014': 2.0,
 'FF_per_capita_2009': 0.00045039999999999994,
 'FF_per_capita_2014': 0.00045,
 'Fast_Food_Restaurants_2009': 4.0,
 'Fast_Food_Restaurants_2014': 4.0,
 'Grocery_2009': 4.0,
 'Grocery_2014': 4.0,
 'Grocery_per_capita_2009': 0.00045039999999999994,
 'Grocery_per_capita_2014': 0.0004503,
 'Limited_Access_2014': 1632.0,
 'Limited_Access_Population_2009': 1617.0,
 'Limited_Access_Populationp_2009': 18.0,
 'Limited_Access_p_2014': 18.0,
 'Median_Household_Income_2009': 58000.0,
 'Median_Household_Income_2014': 68472.0,
 'Population_2009': 8881.0,
 'Population_2014': 8883.0,
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x110d4ba58>,
 'archcard': '1',
 'cartodb_georef_status': 1.0,
 'cartodb_id': 14168.0,
 'city': 'SITKA',
 'drivethru': '1',
 'freewifi': '1',
 'id': 1,
 'invalid_the_geom

In [28]:
state_pop = session.query(Food.Population_2014).all()
state_pop

[(8883.0),
 (32511.0),
 (22177.0),
 (243991.0),
 (243991.0),
 (480446.0),
 (243991.0),
 (243991.0),
 (243991.0),
 (31540.0),
 (484337.0),
 (484337.0),
 (484337.0),
 (46936.0),
 (5812.0),
 (247175.0),
 (247175.0),
 (247175.0),
 (247175.0),
 (247175.0),
 (247175.0),
 (41236.0),
 (18059.0),
 (18059.0),
 (314981.0),
 (74418.0),
 (22663.0),
 (102106.0),
 (22663.0),
 (13920.0),
 (3826.0),
 (3826.0),
 (13920.0),
 (6962.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (19005.0),
 (254272.0),
 (72722.0),
 (19005.0),
 (19005.0),
 (254272.0),
 (659733.0),
 (254272.0),
 (254272.0),
 (72722.0),
 (254272.0),
 (19005.0),
 (254272.0),
 (254272.0),
 (757958.0),
 (757958.0),
 (120303.0),
 (757958.0),
 (757958.0),
 (120303.0),
 (120303.0),
 (208325.0),
 (208325.0),
 (757958.0),
 (120303.0),
 (757958.0),
 (757958.0),
 (757958.0),
 (757958.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (264.0),
 (2

In [29]:
test = session.query(Food).statement
df = pd.read_sql_query(test, session.bind)
df.set_index('id', inplace=True)
st = df.groupby(['state'])['Population_2014'].sum().to_frame()
#the_statess = st.index.tolist()
#the_statess
st

#Median_Household_Income_2014, Food.state, Food.Adult_Diabetes_2014

,Population_2014
state,
AK,41394.0
AL,63934215.0
AR,25262734.0
AZ,732631502.0
CA,791868437.0
CO,77233401.0
CT,62432108.0
DE,3482872.0
FL,788795350.0


In [30]:
result = session.query(Food.Population_2014, Food.Median_Household_Income_2014, Food.state, Food.Adult_Diabetes_2014, Food.Fast_Food_Restaurants_2014).filter_by(state = "AL").all()
record = result[0]
result_dict = {
    "Population_2014": record[0],
    "Median_Household_Income_2014": record[1],
    "state": record[2],
    "Adult_Diabetes_2014": record[3],
    "Fast_Food_Restaurants_2014": record[4],

}
result_dict

{'Adult_Diabetes_2014': 9.5,
 'Fast_Food_Restaurants_2014': 128.0,
 'Median_Household_Income_2014': 46442.0,
 'Population_2014': 154277.0,
 'state': 'AL'}

In [31]:
def state_data(sample):
    sample_name = sample.replace("","")
    result = session.query(Food.Population_2014, Food.Median_Household_Income_2014, Food.state, Food.Adult_Diabetes_2014, Food.Fast_Food_Restaurants_2014).filter_by(state = sample_name).all()
    record = result[0]
    dict_list = {
    "Population_2014": record[0],
    "Median_Household_Income_2014": record[1],
    "state": record[2],
    "Adult_Diabetes_2014": record[3],
    "Fast_Food_Restaurants_2014": record[4],

}
    return (dict_list)
state_data("MA")

{'Adult_Diabetes_2014': 11.1,
 'Fast_Food_Restaurants_2014': 386.0,
 'Median_Household_Income_2014': 59839.0,
 'Population_2014': 555071.0,
 'state': 'MA'}